<a href="https://colab.research.google.com/github/Son-Of-Fry/ProyectoIntegrador/blob/main/Avance4_64Equipo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos alternativos
Está parte del proyecto tiene como objetivo evaluar y comparar el rendimiento de diferentes variantes de los modelos **YOLOv8** y **YOLOv11**, incluyendo sus versiones *n, s, m, l, x*.

El análisis se realiza usando un mismo video de entrada y fragmento temporal, midiendo tanto la velocidad como el uso de recursos computacionales.

Para este proyeto, se lolicitó el desarrollo con Modelo de Yolo, esto por su licencia de uso Libre. El usar otro tipo de Modelos con lincencia o coste, pone en riesgo el objetivo puntual del proyecto.

Para cada modelo, se registrán métricas como:

* **FPS** (cuadros por segundo procesados)
* **Tiempo promedio por frame** (ms)
* **Total de detecciones**
* **Promedio de detecciones por frame**
* **Uso de RAM (GB)**
* **Uso de VRAM promedio y pico (MB)**
* **Uso promedio de GPU (%)**

El objetivo es identificar qué versión ofrece el mejor balance entre rendimiento y eficiencia.


In [ ]:
!nvidia-smi

Sun Oct 19 15:43:21 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.274.02             Driver Version: 535.274.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P0              N/A /  55W |      8MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Entorno de hardware utilizado

El comparativo se ejecutó en un equipo con la siguiente configuración:

* **GPU**: NVIDIA GeForce RTX 4070
* **CUDA Version**: 12.2
* **VRAM disponible**: 8 GB
* **Driver**: 535.274.02
* **Temperatura idle**: ~39°C
* **GPU Utilización inicial**: 0%

Este entorno permite pruebas realistas de modelos de detección en tiempo real y tareas intensivas en computación. Sin embargo, se usará un video recortado para que la prueba sea la misma.

In [ ]:
import cv2
import time
import torch
import psutil
import GPUtil
import pandas as pd
from pynvml import nvmlInit, nvmlShutdown, nvmlDeviceGetHandleByIndex, nvmlDeviceGetPowerUsage, nvmlDeviceGetTemperature, NVML_TEMPERATURE_GPU
from ultralytics import YOLO
from tqdm import tqdm

# --- Inicializar NVML para acceso a temperatura y consumo ---
#Esto activa el acceso a métricas de la GPU (consumo de energía, temperatura, uso de VRAM) usando la interfaz NVML (NVIDIA Management Library).
#Se selecciona la primera GPU (index 0), común en sistemas de una sola tarjeta gráfica.


nvmlInit()
device_handle = nvmlDeviceGetHandleByIndex(0)

#Configuración global
VIDEO_PATH = "zocalo.mp4" #Video del Zocalo de la ciudad de México.
START_SEC = 10
END_SEC = 20
MODEL_VARIANTS = [
    "yolov8n.pt", "yolov8s.pt", "yolov8m.pt", "yolov8l.pt", "yolov8x.pt",
    "yolo11n.pt", "yolo11s.pt", "yolo11m.pt", "yolo11l.pt", "yolo11x.pt",
]

#  Obtener info del video
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
start_frame = int(START_SEC * fps)
end_frame = int(END_SEC * fps)
total_frames = end_frame - start_frame
cap.release()


In [ ]:
# Funciones de medición


#Esta función devuelve la memoria RAM utilizada por el sistema en gigabytes (GB) en el momento de su ejecución.
#Se utiliza para medir el impacto de cada modelo sobre la RAM antes y después del procesamiento.
def get_ram_usage():
    return psutil.virtual_memory().used / (1024**3)


#Esta función recoge cinco métricas clave de la GPU mediante GPUtil y pynvml:
# gpu.memoryUsed:	Memoria de video (VRAM) utilizada en MB
# gpu.memoryUtil:	Porcentaje de uso de memoria de video
# gpu.load:	Carga de procesamiento de la GPU (%)
# power:	Consumo energético en watts (W)
# temp:	Temperatura de la GPU en grados Celsius (°C)


def get_gpu_stats():
    gpus = GPUtil.getGPUs()
    if gpus:
        gpu = gpus[0]#
        power = nvmlDeviceGetPowerUsage(device_handle) / 1000  # mW to W
        temp = nvmlDeviceGetTemperature(device_handle, NVML_TEMPERATURE_GPU) #
        return gpu.memoryUsed, gpu.memoryUtil * 100, gpu.load * 100, power, temp
    return 0, 0, 0, 0, 0

# DF para contabilizar resultados
results_data = []



In [ ]:
#  Probar cada modelo
"""
Para cada modelo se realiza:
	•	Carga del video desde el segundo START_SEC hasta END_SEC.
	•	Carga del modelo YOLO con ultralytics.YOLO.
	•	Inicialización de salida de video para guardar los resultados.
	•	Variables de medición de tiempo, RAM, VRAM, temperatura y potencia.
"""

for model_path in MODEL_VARIANTS:
    print(f"\nEvaluando: {model_path}")#1
    cap = cv2.VideoCapture(VIDEO_PATH)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    model = YOLO(model_path)#2
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') #3
    out_path = f"out_{model_path.replace('.pt', '')}.mp4"
    out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

    total_detections = 0#contador
    start_time = time.time()#temporizador
    ram_before = get_ram_usage()
    vram_usages, gpu_utils, powers, temps = [], [], [], []

    for _ in tqdm(range(total_frames), desc=f"{model_path}"):# Inferencia frame a frame
        ret, frame = cap.read()
        if not ret:
            break

        result = model.predict(source=frame, verbose=False)[0]
        total_detections += len(result.boxes)
        out.write(result.plot())

        vram, vram_util, gpu_util, power, temp = get_gpu_stats()#stats de la gpu
        vram_usages.append(vram)
        gpu_utils.append(gpu_util)
        powers.append(power)
        temps.append(temp)

    end_time = time.time()
    cap.release()
    out.release()

    duration = end_time - start_time
    ram_after = get_ram_usage()

    if total_frames == 0 or duration == 0:
        print("Error: no se procesaron frames.")
        continue
#calculo de métricas de cada frame
    fps_result = total_frames / duration
    ms_per_frame = 1000 * duration / total_frames
    avg_det_per_frame = total_detections / total_frames
    ram_used = ram_after - ram_before
    vram_avg = sum(vram_usages) / len(vram_usages) if vram_usages else 0
    vram_peak = max(vram_usages) if vram_usages else 0
    gpu_util_avg = sum(gpu_utils) / len(gpu_utils) if gpu_utils else 0
    power_avg = sum(powers) / len(powers) if powers else 0
    temp_avg = sum(temps) / len(temps) if temps else 0
#prints mostrando
    print(f"Duración: {duration:.2f} s")
    print(f"FPS: {fps_result:.2f}")
    print(f"Tiempo por frame: {ms_per_frame:.2f} ms")
    print(f"Total detecciones: {total_detections}")
    print(f"Promedio por frame: {avg_det_per_frame:.2f}")
    print(f"RAM usada: {ram_used:.2f} GB")
    print(f"VRAM promedio: {vram_avg:.2f} MB")
    print(f"VRAM pico: {vram_peak:.2f} MB")
    print(f"GPU uso promedio: {gpu_util_avg:.2f} %")
    print(f"Potencia promedio: {power_avg:.2f} W")
    print(f"Temperatura promedio: {temp_avg:.2f} °C")
    print(f"Video guardado: {out_path}")
## añadir al DF de analsis final
    results_data.append({
        "Modelo": model_path,
        "Duración (s)": round(duration, 2),
        "FPS": round(fps_result, 2),
        "ms/frame": round(ms_per_frame, 2),
        "Detecciones totales": total_detections,
        "Detecciones por frame": round(avg_det_per_frame, 2),
        "RAM usada (GB)": round(ram_used, 2),
        "VRAM promedio (MB)": round(vram_avg, 2),
        "VRAM pico (MB)": round(vram_peak, 2),
        "GPU uso promedio (%)": round(gpu_util_avg, 2),
        "Potencia promedio (W)": round(power_avg, 2),
        "Temp promedio (°C)": round(temp_avg, 2)
    })



Evaluando: yolov8n.pt


yolov8n.pt: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 599/599 [01:23<00:00,  7.19it/s]


Duración: 83.31 s
FPS: 7.19
Tiempo por frame: 139.08 ms
Total detecciones: 16416
Promedio por frame: 27.41
RAM usada: 0.40 GB
VRAM promedio: 221.00 MB
VRAM pico: 221.00 MB
GPU uso promedio: 2.62 %
Potencia promedio: 10.27 W
Temperatura promedio: 37.02 °C
Video guardado: out_yolov8n.mp4

Evaluando: yolov8s.pt


yolov8s.pt: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 599/599 [01:28<00:00,  6.78it/s]


Duración: 88.36 s
FPS: 6.78
Tiempo por frame: 147.52 ms
Total detecciones: 20669
Promedio por frame: 34.51
RAM usada: 0.05 GB
VRAM promedio: 271.00 MB
VRAM pico: 271.00 MB
GPU uso promedio: 3.70 %
Potencia promedio: 10.54 W
Temperatura promedio: 36.00 °C
Video guardado: out_yolov8s.mp4

Evaluando: yolov8m.pt


...

Duración: 45.22 s
FPS: 13.25
Tiempo por frame: 75.49 ms
Total detecciones: 19630
Promedio por frame: 32.77
RAM usada: 0.31 GB
VRAM promedio: 1037.00 MB
VRAM pico: 1037.00 MB
GPU uso promedio: 17.06 %
Potencia promedio: 13.28 W
Temperatura promedio: 38.07 °C
Video guardado: out_yolo11m.mp4

Evaluando: yolo11l.pt


yolo11l.pt: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 599/599 [01:28<00:00,  6.76it/s]


Duración: 88.57 s
FPS: 6.76
Tiempo por frame: 147.86 ms
Total detecciones: 19806
Promedio por frame: 33.07
RAM usada: 0.27 GB
VRAM promedio: 1135.00 MB
VRAM pico: 1135.00 MB
GPU uso promedio: 7.93 %
Potencia promedio: 11.80 W
Temperatura promedio: 37.38 °C
Video guardado: out_yolo11l.mp4

Evaluando: yolo11x.pt


yolo11x.pt: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 599/599 [01:27<00:00,  6.84it/s]

Duración: 87.63 s
FPS: 6.84
Tiempo por frame: 146.29 ms
Total detecciones: 20683
Promedio por frame: 34.53
RAM usada: -0.21 GB
VRAM promedio: 1441.00 MB
VRAM pico: 1441.00 MB
GPU uso promedio: 14.18 %
Potencia promedio: 13.59 W
Temperatura promedio: 37.30 °C
Video guardado: out_yolo11x.mp4


In [ ]:

# --- Mostrar tabla final ---
df = pd.DataFrame(results_data)
print("\nResultados comparativos:\n")
print(df.to_string(index=False))
# Finalizar NVML
nvmlShutdown()



Resultados comparativos:

    Modelo  Duración (s)   FPS  ms/frame  Detecciones totales  Detecciones por frame  RAM usada (GB)  VRAM promedio (MB)  VRAM pico (MB)  GPU uso promedio (%)  Potencia promedio (W)  Temp promedio (°C)
yolov8n.pt         83.31  7.19    139.08                16416                  27.41            0.40               221.0           221.0                  2.62                  10.27               37.02
yolov8s.pt         88.36  6.78    147.52                20669                  34.51            0.05               271.0           271.0                  3.70                  10.54               36.00
yolov8m.pt         88.64  6.76    147.98                23120                  38.60            0.07               421.0           421.0                  7.10                  11.43               36.00
yolov8l.pt         88.99  6.73    148.56                24057                  40.16            0.04               597.0           597.0                 11.06       

Conclusiones del comparativo:

1.	Precisión: YOLOv8x.pt y YOLO11x.pt lograron más de 26,000 detecciones, siendo los más precisos. Las versiones pequeñas (v8n, v11n) detectan menos, como era esperado.
2.	Rendimiento: Modelos ligeros (n, s) fueron los más rápidos (7.5 FPS), mientras que los más grandes (x) fueron más lentos (6.5 FPS) por su complejidad.
3.	Recursos: El uso de VRAM y GPU crece con el tamaño del modelo. La RAM se mantiene estable. El consumo de energía y temperatura fue bajo y constante.

4.	Caso destacado: YOLOv11m.pt duplicó el rendimiento esperado (13.25 FPS), lo que sugiere una optimización eficiente para una PC (RTX 4070).